In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy, os


# Global Climate Change

## Background
Climate Change is defined as the long-term changes in Earth's climate and weather patterns and Climate is often defined as the average weather at a particular place or as the mean state and variablility of features such as temperature, precipitiation, and humidity over some extended time period. Shifts in the climate can be natural, such as changes in the sun's activity or it can be anthropogenic. The extent of the current climate change can laragly be attributed to human actvity such as the burning of fossil fuels like coal, oil, and gas. The burning of fossil fuels generates greenhouse gas which traps the sun's heat and therefore rasies the temperature. The average tempertaure of the Earth's surface is now about 1.1 C warm than it was before the indusrial revolution. Climate change means more than just warmer temperatures, it also includes, among others, intense droughts, severe fire, rising sea level, and more intense storms.

## Goals
The goal of this project is to examine the temperature and precipitation aspects of climate data to understand global and regional trends and the realtion between climate varaibles and greenhouse gas emissions. Futhermore, it aims to model potential future scenarios based on current trends over differnt time horizen.
    
1. Are there discerbinle trends in temperature and precipitation over the given time period, globally and across different regions/countries?
2. Are there regions/countries that are experiencing more drastic changes compared to others?
3. Are there any extreme weather events that show a significant increase in frequency or itensity over time
4. Are there any discerbinle trends in temperature and precipitation that differs between developed and developing countries?
5. Can the data be compared with greenhouse gas emissions data to assess the realtionship between emissions and climate varaibles?
6. If the current trends shown in the data continues without any significant change, where would we be in 10/50/100 years from now?


# Data Gathering
## Dataset
The main dataset used in this project is the Global Historical Climatology Network - Daily (GHCN-Daily), Version 3 sourced from the National Oceanic and Atmospheric Administration. The dataset is available for public use , with the only limitations being that the "*NOAA and NCEI cannot provide any warranty as to the accuracy, reliability, or completeness of furnished data. Users assume responsibility to determine the usability of these data. The user is responsible for the results of any application of this data for other than its intended purpose.*"

### Dataset Metadata
File Size - 131 GB  <br>
Number of Files - 125, 391

#### Measurements:
1. Precipitations (tenths of mm, PRCP)
2. Temperature Max (tenths of degrees C, TMAX)
3. Temperature Min (tenths of degrees C, TMIN)
4. Temperature Average (tenths of degrees C, TAXN)
5. Snowfall (mm, SNOW)
6. Snow Depth (mm SNWD)

### Data Range
>Most Frequent Start Date: 1901-01-01      *(count: 1715)* <br>
>Most Frequent End Date: 2024-02-08    *(count: 11137)*
>
>
>Date Range: 1901-01-01 ~ 2023-12-31

# Data Cleaning

## Filtering Out Irrelevant and Empty Columns

In [ ]:

def filter_csv(input_file, columns):
    df_in = pd.read_csv(input_file, dtype=str)
    df_columns = df_in.columns.to_list()

    valid_col = [col for col in columns if col in df_columns]

    #Filter the DataFrame based on columns
    filtered_df = df_in[valid_col]

    return filtered_df

def check_empty_columns(df):
    #Filters out the empty columns
    filtered_df = df.dropna(axis=1, how="all")

    return filtered_df


def filter_columns():
    #columns names in csv file
    col = ["TMAX", 'TMIN', 'TAXN', 'PRCP', 'SNOW', 'SNWD', 'DATE', 'STATION']

    input_loc = r'Data\\'
    output_loc = r'Filtered Data'

    #filter file in dir to perfect & other folder based contents
    for item in os.listdir(input_loc):
        df = filter_csv(input_loc+ item, col)
        
        filter_df = check_empty_columns(df)

        filter_df.to_csv(os.path.join(output_loc, item), index=False)


## Filtering Out Files Without Requisite Data

In [ ]:
def filter_columns(df):
    temp_col = ["TMAX", "TMIN"]
    avgTemp_col = ["TAXN"]
    other_col = ["PRCP", 'SNOW', 'SNWD']

    df_col = df.columns.to_list()

    temp_cond = all(col in df_col for col in temp_col)
    avgTemp_cond = all(col in df_col for col in avgTemp_col)

    if temp_cond or avgTemp_cond:
        if all(col in df_col for col in other_col): return 'pt' #add to perfect data
        else: return 'tp' #add to temp data
    else: return 'pr' #only precipitation data

def run_filter():
    #columns names in csv file
    col = ["TMAX", 'TMIN', 'TAXN', 'PRCP', 'SNOW', 'SNWD', 'DATE', 'STATION']

    input_loc = r'Data\\'
    output_loc = r'Filtered Data'

    #filter file in dir to perfect & other folder based contents
    for item in os.listdir(input_loc):
        filter_df = pd.read_csv(input_loc+item, dtype=str)

        key = filter_columns(filter_df)

        # check the key to see which folder to put csv file in Filtered Data
        if key == 'pt': filter_df.to_csv(os.path.join(output_loc, 'Perfect', item), index=False)
        if key == 'tp': filter_df.to_csv(os.path.join(output_loc, 'Temperature', item), index=False)
        if key == 'pr': filter_df.to_csv(os.path.join(output_loc, 'Precipitation', item), index=False)

## Filtering out Data Over Defined Threshold

## Handling Missing Data

## Data Aggregation (Temporal and Spatial)

## Data Comparison

# Data Exploration and Feature Engineering

### Trends
Are there discerbinle trends in temperature and precipitation over the given time period, globally and across different regions/countries?
Are there any extreme weather events that show a significant increase in frequency or itensity over time

### Regional Trend Comparison
Are there regions/countries that are experiencing more drastic changes compared to others?
Are there any discerbinle trends in temperature and precipitation that differs between developed and developing countries?

### Data Comparison
Can the data be compared with greenhouse gas emissions data to assess the realtionship between emissions and climate varaibles?

# Model Development

If the current trends shown in the data continues without any significant change, where would we be in 10/50/100 years from now?

# Model Evaluation and Selection

# Conclusion